# PyPDFLoader

This notebook provides a quick overview for getting started with `PyPDF` [document loader](https://python.langchain.com/v0.2/docs/concepts/#document-loaders). For detailed documentation of all DocumentLoader features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html).


## Overview
### Integration details


| Class | Package | Local | Serializable | JS support|
| :--- | :--- | :---: | :---: |  :---: |
| [PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html) | [langchain_community](https://api.python.langchain.com/en/latest/community_api_reference.html) | ✅ | ❌ | ❌ | 
### Loader features
| Source | Document Lazy Loading | Native Async Support
| :---: | :---: | :---: | 
| PyPDFLoader | ✅ | ❌ | 

## Setup

### Credentials

No credentials are required to use `PyPDFLoader`.

### Installation

To use `PyPDFLoader` you need to have the `langchain-community` python package downloaded:

In [ ]:
%pip install -qU langchain_community pypdf

## Initialization

Now we can instantiate our model object and load documents:

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "./example_data/layout-parser-paper.pdf",
)

## Load

In [2]:
docs = loader.load()
docs[0]

Document(metadata={'source': './example_data/layout-parser-paper.pdf', 'page': 0}, page_content='LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to 

In [3]:
print(docs[0].metadata)

{'source': './example_data/layout-parser-paper.pdf', 'page': 0}


## Lazy Load


In [4]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        pages = []
len(pages)

6

In [5]:
print(pages[0].page_content[:100])
print(pages[0].metadata)

LayoutParser : A Uniﬁed Toolkit for DL-Based DIA 11
focuses on precision, eﬃciency, and robustness. 
{'source': './example_data/layout-parser-paper.pdf', 'page': 10}


## API reference

For detailed documentation of all `PyPDFLoader` features and configurations head to the API reference: https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html